# Case study of June 28, 2021

In [18]:
%store -r city_center clipped_upwind_wedge clipped_downwind_wedge

In [16]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [4]:
tracked_cells = xr.open_dataset('C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/stats/trackstats_20210601.0000_20210630.2359.nc')

In [5]:
# Define your time range
start_time = np.datetime64('2021-06-28T00:00:00')
end_time = np.datetime64('2021-06-28T23:59:59')

# Create a mask where base_time is within the specified range
time_mask = (tracked_cells.start_basetime >= start_time) & (tracked_cells.start_basetime <= end_time)

# Apply this mask across the 'tracks' dimension
filtered_ds = tracked_cells.where(time_mask, drop=True)
filtered_ds

<xarray.Dataset>
Dimensions:                  (tracks: 1063, times: 100)
Coordinates:
  * tracks                   (tracks) int64 9136 9137 9138 ... 10196 10197 10198
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) float64 2.0 7.0 6.0 4.0 ... 6.0 10.0 7.0
    base_time                (tracks, times) datetime64[ns] 2021-06-28T00:05:...
    meanlat                  (tracks, times) float32 30.01 30.02 nan ... nan nan
    meanlon                  (tracks, times) float32 -95.16 -95.18 ... nan nan
    area                     (tracks, times) float32 22.0 28.5 nan ... nan nan
    cloudnumber              (tracks, times) float64 23.0 17.0 nan ... nan nan
    ...                       ...
    start_split_tracknumber  (tracks) float64 9.133e+03 nan nan ... nan nan nan
    start_split_timeindex    (tracks) float64 0.0 nan nan nan ... nan nan nan
    start_split_cloudnumber  (tracks) float64 27.0 nan nan nan ... nan nan nan
    end_merge_tracknumber    (tracks) float64 9.126e+03 nan nan ... nan nan nan
    end_merge_timeindex      (tracks) float64 5.0 nan nan nan ... nan nan nan
    end_merge_cloudnumber    (tracks) float64 3.0 nan nan nan ... nan nan nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

## Extract storms in the region

### Urban area

In [21]:
# Initiation at urban area
def lat_lon_to_cartesian(lat, lon, R=6371):
    x = R * np.radians(lon)
    y = R * np.radians(lat)
    return x, y

start_lon = filtered_ds['cell_meanlon'].isel(times=0)
start_lat = filtered_ds['cell_meanlat'].isel(times=0)
start_hour = filtered_ds['start_basetime'].dt.hour
hour_bin = np.arange(0, 25, 1)

center_lat, center_lon, radius = city_center[1], city_center[0], 76.35
center_x, center_y = lat_lon_to_cartesian(center_lat, center_lon)
storm_x, storm_y = lat_lon_to_cartesian(start_lat, start_lon)
distances = np.sqrt((storm_x - center_x)**2 + (storm_y - center_y)**2)
storms_in_circle = start_hour[distances <= radius]
hist_storms_in_circle, bins = np.histogram(storms_in_circle, bins=hour_bin, range=(0, 24), density=False)
hist_storms_in_circle_LT = np.roll(hist_storms_in_circle, -6)
storms_in_circle
# Fraction of E.SDC tracks to all tracks
#hist_storms_in_circle_LT_frac = 100 * (hist_storms_in_circle_LT / hist_starthour_LT)

<xarray.DataArray 'hour' (tracks: 323)>
array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,
        5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16,
       16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22],
      dtype=int64)
Coordinates:
  * tracks   (tracks) int64 9136 9141 9142 9145 9147 ... 10161 10180 10182 10185
    times    int64 0
Attributes:
    long_name:  Start Epoch time of each track

### Upwind area

In [20]:
storm_points = [Point(lon, lat) for lon, lat in zip(start_lon, start_lat)]
storms_in_upwind_wedge = [start_hour[i] for i, point in enumerate(storm_points) if clipped_upwind_wedge.contains(point)]
storms_in_upwind_wedge = xr.concat(storms_in_upwind_wedge, dim='tracks')
hist_storms_in_upwind_wedge, bins = np.histogram(storms_in_upwind_wedge, bins=hour_bin, range=(0, 24), density=False)
storms_in_upwind_wedge

<xarray.DataArray 'hour' (tracks: 452)>
array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,
        7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
       22, 22, 23, 23, 23, 23, 23, 23, 23, 23], dtype=int64)
Coordinates:
  * tracks   (tracks) int64 9140 9142 9143 9144 9146 ... 10192 10193 10196 10198
Attributes:
    long_name:  Start Epoch time of each track

### Downwind area

In [22]:
storm_points = [Point(lon, lat) for lon, lat in zip(start_lon, start_lat)]
storms_in_downwind_wedge = [start_hour[i] for i, point in enumerate(storm_points) if clipped_downwind_wedge.contains(point)]
storms_in_downwind_wedge = xr.concat(storms_in_downwind_wedge, dim='tracks')
hist_storms_in_downwind_wedge, bins = np.histogram(storms_in_downwind_wedge, bins=hour_bin, range=(0, 24), density=False)
storms_in_downwind_wedge

<xarray.DataArray 'hour' (tracks: 22)>
array([ 0,  0,  2,  3,  6,  7,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9, 10,
       10, 12, 12, 15, 17], dtype=int64)
Coordinates:
  * tracks   (tracks) int64 9164 9173 9268 9294 9421 ... 9706 9751 9903 9983
Attributes:
    long_name:  Start Epoch time of each track

In [34]:
circle = storms_in_circle.tracks.values
stats_circle = filtered_ds.sel(tracks=circle)

upwind = storms_in_upwind_wedge.tracks.values
stats_upwind = filtered_ds.sel(tracks=upwind)

downwind = storms_in_downwind_wedge.tracks.values
stats_downwind = filtered_ds.sel(tracks=downwind)

In [35]:
stats_downwind

<xarray.Dataset>
Dimensions:                  (tracks: 22, times: 100)
Coordinates:
  * tracks                   (tracks) int64 9164 9173 9268 ... 9751 9903 9983
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) float64 2.0 5.0 2.0 2.0 ... 2.0 2.0 1.0
    base_time                (tracks, times) datetime64[ns] 2021-06-28T00:27:...
    meanlat                  (tracks, times) float32 29.6 29.62 nan ... nan nan
    meanlon                  (tracks, times) float32 -96.21 -96.21 ... nan nan
    area                     (tracks, times) float32 14.0 14.75 nan ... nan nan
    cloudnumber              (tracks, times) float64 35.0 34.0 nan ... nan nan
    ...                       ...
    start_split_tracknumber  (tracks) float64 nan 8.937e+03 ... nan 9.867e+03
    start_split_timeindex    (tracks) float64 nan 39.0 nan nan ... nan nan 22.0
    start_split_cloudnumber  (tracks) float64 nan 4.0 nan nan ... nan nan 7.0
    end_merge_tracknumber    (tracks) float64 nan nan nan nan ... nan nan nan
    end_merge_timeindex      (tracks) float64 nan nan nan nan ... nan nan nan
    end_merge_cloudnumber    (tracks) float64 nan nan nan nan ... nan nan nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

In [44]:
filtered_ds.sel(tracks=9155)

<xarray.Dataset>
Dimensions:                  (times: 100)
Coordinates:
    tracks                   int64 9155
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           float64 10.0
    base_time                (times) datetime64[ns] 2021-06-28T00:21:40.81400...
    meanlat                  (times) float32 29.23 29.23 29.23 ... nan nan nan
    meanlon                  (times) float32 -95.54 -95.56 -95.57 ... nan nan
    area                     (times) float32 18.75 16.75 22.75 ... nan nan nan
    cloudnumber              (times) float64 31.0 32.0 24.0 24.0 ... nan nan nan
    ...                       ...
    start_split_tracknumber  float64 nan
    start_split_timeindex    float64 nan
    start_split_cloudnumber  float64 nan
    end_merge_tracknumber    float64 nan
    end_merge_timeindex      float64 nan
    end_merge_cloudnumber    float64 nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

In [56]:
tracks_ds1 = stats_upwind.tracks.values
tracks_ds2 = stats_circle.tracks.values

# Find common tracks
common_tracks = set(tracks_ds1).intersection(tracks_ds2)


### Calculate the cell development stages

In [57]:
# Load the dataset (replace with your dataset loading line if different)
ds = stats_circle

# Extract maxETH_20dbz and calculate the rate of change
maxETH_20dbz = ds['maxETH_20dbz']
rate_of_change = maxETH_20dbz.diff(dim='times')
padded_rate_of_change = xr.concat([xr.full_like(maxETH_20dbz.isel(times=0), np.nan), rate_of_change], dim='times')

# Calculate percentiles for thresholds and classify stages
percentile_25, percentile_75 = np.nanpercentile(padded_rate_of_change, [25, 75])
stages = xr.where(padded_rate_of_change >= percentile_75, 1, np.nan)  # Developing
stages = xr.where((padded_rate_of_change > percentile_25) & (padded_rate_of_change < percentile_75), 2, stages)  # Mature
stages = xr.where(padded_rate_of_change <= percentile_25, 3, stages)  # Dissipating
ds['cell_stages'] = stages.fillna(0)  # Handling initial NaNs

# Final dataset with cloud cell stages classification
final_result = ds
